In [1]:
import os
import warnings
from datetime import datetime
from zoneinfo import ZoneInfo  # on windows need tzdata package

import numpy as np
import plot_utils as pu
from astropy import units as u
from astropy.coordinates import ICRS, AltAz, EarthLocation, SkyCoord
from astropy.table import QTable, unique
from astropy.time import Time
from astropy.visualization.wcsaxes.frame import EllipticalFrame
from astropy.wcs import WCS
from astroquery.exceptions import NoResultsWarning
from astroquery.simbad import Simbad
from matplotlib import patheffects
from matplotlib import pyplot as plt
from matplotlib.colors import LinearSegmentedColormap, LogNorm
from timezonefinder import TimezoneFinder
from wand.image import Image

tf = TimezoneFinder()
backgroundcolour = "#171726"
# backgroundcolour = "#000"
foregroundcolour = "#FFF"

cmap = LinearSegmentedColormap.from_list("sky", [backgroundcolour, foregroundcolour])

Simbad.reset_votable_fields()

generic_maximum_magnitude = 10

## User Inputs

In [2]:
location = "Toronto"
observation_point = (
    45 * u.deg,
    160 * u.deg,
)  # altitude (deg from horizon), azimuth (eastwards from north)


view_radius = 15 * u.deg
maximum_magnitude = 4

observation_start = {
    "year": 2025,
    "month": 2,
    "day": 18,
    "hour": 18,
    "minute": 0,
    "second": 0,
}
observation_length = 2 * u.hour
observation_frequency = 10 * u.minute

image_pixels = 250
fps = 2
image_directory = os.getcwd()
gif_fname = f"{image_directory}/SkySim.gif"
delete_frames = True

In [14]:
# query to a lat/long
earth_location = EarthLocation.of_address(location)
lat, lon = [l.to(u.deg).value for l in [earth_location.lat, earth_location.lon]]
timezone = ZoneInfo(tf.timezone_at(lat=lat, lng=lon))

frames = np.ceil((observation_length / observation_frequency).decompose()).astype(int)

dt_native_start = datetime(**observation_start, tzinfo=timezone)
dt_astropy_start = Time(dt_native_start).to_value("mjd")
dt_astropy_end = dt_astropy_start + observation_length.to(u.day).value
dt_astropy = Time(
    np.arange(dt_astropy_start, dt_astropy_end, observation_frequency.to(u.day).value),
    format="mjd",
)

## Convert Observing Point to RA/Dec

In [15]:
# generate a coordinate frame for the observation
earth_frame = AltAz(
    obstime=dt_astropy,
    az=observation_point[1],
    alt=observation_point[0],
    location=earth_location,
)

# perform the conversion
ra_dec = SkyCoord(earth_frame.transform_to(ICRS()))

In [16]:
print(ra_dec.to_string("hmsdms"))
print(earth_frame[0])
# ra_dec = SkyCoord(ra='5h36m12.81s',dec='-1d12m6.91s')

['04h33m46.38070874s +00d22m11.67558581s', '04h43m47.96286255s +00d22m32.24492159s', '04h53m49.54594336s +00d22m53.14399712s', '05h03m51.13007289s +00d23m14.33282531s', '05h13m52.71537134s +00d23m35.77086435s', '05h23m54.30195714s +00d23m57.41709528s', '05h33m55.88994665s +00d24m19.23010035s', '05h43m57.47945523s +00d24m41.16814204s', '05h53m59.07059281s +00d25m03.18924336s', '06h04m00.66346777s +00d25m25.25126763s', '06h14m02.25818522s +00d25m47.31199918s', '06h24m03.85484678s +00d26m09.32922406s', '06h34m05.45355028s +00d26m31.26081082s']
<AltAz Coordinate (obstime=60724.958333333336, location=(851522.3283809287, -4543023.914790087, 4380315.422523561) m, pressure=0.0 hPa, temperature=0.0 deg_C, relative_humidity=0.0, obswl=1.0 micron): (az, alt) in deg
    (160., 45.)>


## Query SIMBAD to get a catalogue of objects with relevant data

In [17]:
Simbad.reset_votable_fields()
Simbad.add_votable_fields("otype", "V", "ids")
criteria = f"otype != 'err' AND V < {generic_maximum_magnitude}"
query_result = QTable(
    Simbad.query_region(ra_dec, radius=view_radius, criteria=criteria)
)

In [18]:
# clean up the result
columns_to_remove = [
    "coo_err_min",
    "coo_err_angle",
    "coo_wavelength",
    "coo_bibcode",
    "coo_err_maj",
]
for colname in columns_to_remove:
    query_result.remove_column(colname)

# rename columns
query_result.rename_column("main_id", "id")
query_result.rename_column("otype", "object_type")
query_result.rename_column("V", "magnitude")

# round columns
query_result["ra"] = query_result["ra"].round(5)
query_result["dec"] = query_result["dec"].round(5)
query_result["magnitude"] = query_result["magnitude"].round(3)

# create human-readable name column
query_result["ids_list"] = [i.split("|") for i in query_result["ids"]]
names_column = []
for id, namelist in zip(query_result["id"].data, query_result["ids_list"].data):
    item_names = [n[5:] for n in namelist if "NAME" in n]
    if len(item_names) == 0:
        names_column.append(id)
    elif len(item_names) == 1:
        names_column.append(item_names[0])
    else:
        names_column.append("/".join(item_names))
query_result["name"] = names_column
query_result.remove_columns(["ids", "ids_list"])

## Filter data

In [19]:
# only keep items with magnitudes less than the given maximum
query_result = query_result[query_result["magnitude"] < maximum_magnitude]

In [20]:
# remove child elements
parents = query_result["id"]  # check all items, regardless of type
parents_string = tuple(parents.data)
parent_query_adql = f"""
    SELECT main_id AS "child_id",
    parent_table.id AS "parent_id"
    FROM (SELECT oidref, id FROM ident WHERE id IN {parents_string}) AS parent_table,
    basic JOIN h_link ON basic.oid = h_link.child
    WHERE h_link.parent = parent_table.oidref;
"""
with warnings.catch_warnings(action="ignore", category=NoResultsWarning):
    hierarchies = Simbad.query_tap(parent_query_adql)
children = unique(hierarchies)["child_id"].data
query_result.add_index("id")
for child_id in children:
    if child_id in query_result["id"]:
        query_result.remove_rows(query_result.loc_indices[child_id])

query_result = unique(query_result)

## Calculate image parameters

In [21]:
query_result["flux"] = 10 ** (
    -query_result["magnitude"] / 2.5
)  # relative to V-band reference flux
query_result["flux"] = query_result["flux"].round(5)

degrees_per_pixel = (view_radius / (image_pixels / 2)).to(u.deg).value

airy_disk_minimum = 23 * u.arcmin / 2  # based on Vega spread in SIMBAD image
airy_disk_pixels = airy_disk_minimum.to(u.deg) / (degrees_per_pixel * u.deg)

# assume the airy disk is at 3x standard deviation of the Gaussian
std_dev = airy_disk_pixels / 3

# standard deviation is "diameter" whilst airy is "radius"
std_dev *= 2

query_result["scaled_flux"] = np.log10(query_result["flux"])
query_result["scaled_flux"] = (
    query_result["scaled_flux"] - np.min(query_result["scaled_flux"]) + 0.2
)
query_result["scaled_flux"] /= np.max(query_result["scaled_flux"])
query_result["scaled_flux"] = query_result["scaled_flux"].round(5)


def get_intensity(radius, flux, sigma):
    """
    How much light is observed from a star at some radius away from it
    """
    exponential = np.exp(-(radius**2) / (sigma**2))

    return flux * exponential

In [22]:
query_result.sort("magnitude")
query_result

id,ra,dec,magnitude,object_type,name,flux,scaled_flux
,deg,deg,,,,,
object,float64,float64,float64,object,str71,float64,float64
* bet Ori,78.63447,-8.20164,0.13,s*b,Rigel,0.88716,1.0
* alf Ori,88.79294,7.40706,0.42,s*r,Betelgeuse,0.6792,0.93302
* gam Ori,81.28276,6.3497,1.64,V*,Bellatrix,0.2208,0.65127
* eps Ori,84.05339,-1.20192,1.69,s*b,Alnilam,0.21086,0.63972
* zet Ori,85.18969,-1.94257,1.77,**,Alnitak,0.19588,0.62125
* kap Ori,86.93912,-9.6696,2.06,s*b,Saiph,0.14997,0.55428
* del Ori,83.00167,-0.2991,2.41,**,Mintaka,0.10864,0.47344
NGC 1980,83.81,-5.924,2.5,OpC,Lower Sword,0.1,0.45266


## Add stars to image

In [23]:
image = np.zeros((frames, image_pixels, image_pixels))

maxradius = np.ceil(10 * std_dev)  # calculate contribution out to 5 standard deviations
radius_vector = np.arange(-maxradius, maxradius + 1).astype(int)
area = np.array(
    np.meshgrid(radius_vector, radius_vector)
)  # mesh of points which will map to around the star

radial_distance = np.sqrt(
    area[0] ** 2 + area[1] ** 2
)  # radius measurement at each of the meshgrid points

unique_radii = np.unique(radial_distance)

# all of the locations where each unique radius is found
radius_locations = [
    np.array(np.where(radial_distance == radius)).T for radius in unique_radii
]

wcs_by_frame = []
for i in range(frames):
    wcs = WCS(naxis=2)
    wcs.wcs.crpix = [image_pixels / 2] * 2
    wcs.wcs.cdelt = [degrees_per_pixel, degrees_per_pixel]
    wcs.wcs.crval = [ra_dec[i].ra.value, ra_dec[i].dec.value]
    wcs.wcs.ctype = ["RA", "DEC"]
    wcs.wcs.cunit = [u.deg, u.deg]
    wcs_by_frame.append(wcs)

for name, ra, dec, flux in query_result[["name", "ra", "dec", "scaled_flux"]]:
    skycoord = SkyCoord(ra=ra, dec=dec)  # star location

    xy_by_frame = [np.round(skycoord.to_pixel(wcs)).astype(int) for wcs in wcs_by_frame]

    # do all points at some given radius at once
    for radius, points in zip(unique_radii, radius_locations):
        brightness = get_intensity(radius, flux, std_dev)

        for area_x, area_y in points:
            for i, (x, y) in enumerate(xy_by_frame):
                # get the pixel locations where this radius applies
                x_ = x + area[1][area_x, area_y]
                y_ = y + area[0][area_x, area_y]

                if (0 <= x_ < image.shape[2]) and (0 <= y_ < image.shape[1]):
                    image[i, y_, x_] += brightness

## Plot result

In [24]:
gif_frame_paths = []
for i, (frametime, wcs) in enumerate(zip(dt_astropy, wcs_by_frame)):
    fig, ax = pu.fig_setup(wcs=wcs, plot_kwargs={"frame_class": EllipticalFrame})

    im = ax.imshow(image[i], vmin=0, vmax=1, cmap=cmap)

    pu.style_wcs_axes(ax, axis_ticks=(False, False))
    ax.invert_xaxis()

    utc_datetime = frametime.to_datetime().replace(tzinfo=ZoneInfo("UTC"))
    local_datetime = utc_datetime.astimezone(tz=timezone)

    location_string = f"{location}"
    datetime_string = local_datetime.strftime("%Y-%m-%d %H:%M %Z")
    altaz_string = f"Altitude: {observation_point[0].to_string(format="latex")}, Azimuth: {observation_point[1].to_string(format="latex")}, FOV: {(2*view_radius).to_string(format="latex")}"
    ax.set_title(f"{location_string}\n{datetime_string}\n{altaz_string}")

    fname = f"SkySim_{i}.png"
    pu.save_fig(fig, fname, directory=image_directory)
    gif_frame_paths.append(fname)

/home/taiwithers/projects/skysim/mvp/SkySim_0.png saved.
/home/taiwithers/projects/skysim/mvp/SkySim_1.png saved.
/home/taiwithers/projects/skysim/mvp/SkySim_2.png saved.
/home/taiwithers/projects/skysim/mvp/SkySim_3.png saved.
/home/taiwithers/projects/skysim/mvp/SkySim_4.png saved.
/home/taiwithers/projects/skysim/mvp/SkySim_5.png saved.
/home/taiwithers/projects/skysim/mvp/SkySim_6.png saved.
/home/taiwithers/projects/skysim/mvp/SkySim_7.png saved.
/home/taiwithers/projects/skysim/mvp/SkySim_8.png saved.
/home/taiwithers/projects/skysim/mvp/SkySim_9.png saved.
/home/taiwithers/projects/skysim/mvp/SkySim_10.png saved.
/home/taiwithers/projects/skysim/mvp/SkySim_11.png saved.


In [25]:
gif_sequence = Image()
for fname in gif_frame_paths:
    frame = Image(filename=fname)
    frame.delay = int(100 / fps)
    gif_sequence.sequence.append(frame)

gif_sequence.save(filename=gif_fname)

In [26]:
if delete_frames:
    for fname in gif_frame_paths:
        os.remove(fname)